In [13]:
import pandas as pd
# import matplotlib.pyplot as plt
import nltk
import time, os, pickle
from nltk.tokenize import word_tokenize, sent_tokenize
from itertools import combinations, permutations
# plt.style.use('ggplot')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# import data

In [2]:
df1 = pd.read_csv(r'../all-the-news/articles1.csv')
df2 = pd.read_csv(r'../all-the-news/articles2.csv')
df3 = pd.read_csv(r'../all-the-news/articles3.csv')
df = df1.append(df2).append(df3)
df.index = range(df.shape[0])
del df1, df2, df3
df.shape

(142570, 10)

In [3]:
df.sample(5)

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
99967,103426,151867,Manchester United’s Juan Mata delivers derby d...,Guardian,Paul Wilson,2016-10-26,2016.0,10.0,https://www.theguardian.com/football/2016/oct/...,Manchester United’s week took a much needed tu...
101096,104556,153340,MLS All-Stars 1-2 Arsenal – as it happened,Guardian,Graham Parker,2017-02-20,2017.0,2.0,https://www.theguardian.com/football/live/2016...,"Arsenal win it at the death then, with a few t..."
16672,16673,35410,Wall Street Journal: Twitter Engulfed in Media...,Breitbart,Breitbart News,2016-01-11,2016.0,1.0,NaN,Yoree Koh writes in the Wall Street Journal:[T...
109643,113105,167812,The Strange And Surprising Debate Over How To ...,NPR,Emily Sohn,2016-03-13,2016.0,3.0,http://www.npr.org/sections/goatsandsoda/2016/...,It used to be common knowledge: Malnourished k...
96919,100378,147855,La La Land wins best film at Directors Guild a...,Guardian,NaN,2017-02-05,2017.0,2.0,https://www.theguardian.com/film/2017/feb/05/l...,La La Land swept up another major Hollywood pr...


In [4]:
len(df.content[0])

5607

In [5]:
df.content[0]

'WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been 

In [6]:
df.publication.unique()

array(['New York Times', 'Breitbart', 'CNN', 'Business Insider',
       'Atlantic', 'Fox News', 'Talking Points Memo', 'Buzzfeed News',
       'National Review', 'New York Post', 'Guardian', 'NPR', 'Reuters',
       'Vox', 'Washington Post'], dtype=object)

# Define bias of each publication based on mediabiasfactcheck.com

In [7]:
# actual labels from MBFC
bias_dict = {
    'New York Times': 'left-center',
    'Breitbart': 'extreme-right',
    'CNN': 'left',
    'Business Insider': 'left-center',
    'Atlantic': 'left-center',
    'Fox News': 'right',
    'Talking Points Memo': 'left',
    'Buzzfeed News': 'left-center',
    'National Review': 'right',
    'New York Post': 'right-center',
    'Guardian': 'left-center',
    'NPR': 'left-center',
    'Reuters': 'neutral',
    'Vox': 'left',
    'Washington Post': 'left-center'
}

In [8]:
# simplified labels
for k,v in bias_dict.items():
    if 'left' in v:
        bias_dict[k] = 'liberal'
    elif 'right' in v:
        bias_dict[k] = 'conservative'
    else:
        bias_dict[k] = 'neutral'
for k,v in bias_dict.items():
    print(k,v)

New York Times liberal
Breitbart conservative
CNN liberal
Business Insider liberal
Atlantic liberal
Fox News conservative
Talking Points Memo liberal
Buzzfeed News liberal
National Review conservative
New York Post conservative
Guardian liberal
NPR liberal
Reuters neutral
Vox liberal
Washington Post liberal


In [9]:
!pwd

/home/alex/Documents/MIDS/w266/final_project/w266_final_project


In [16]:
# load in the bigrams and lexicons
datapath = r'../data'
with open(os.path.join(datapath, 'libCons.pickle'), 'rb') as f:
    libCons = pickle.load(f)
with open(os.path.join(datapath, 'conCons.pickle'), 'rb') as f:
    conCons = pickle.load(f)
with open(os.path.join(datapath, 'libGrams.pickle'), 'rb') as f:
    libGrams = pickle.load(f)
with open(os.path.join(datapath, 'conGrams.pickle'), 'rb') as f:
    conGrams = pickle.load(f)

# testing

In [17]:
df.title[0]

'House Republicans Fret About Winning Their Health Care Suit - The New York Times'

In [18]:
sentences = sent_tokenize(df.content[0])
sentences.append(df.title[0])
len(sentences)

27

In [60]:
s = sent_tokenize(df.content[0])
w = word_tokenize(s[0])
bigrams = []
for i in range(len(w) - 1):
    bigrams.append(' '.join((w[i], w[i+1])))
bigrams

['WASHINGTON —',
 '— Congressional',
 'Congressional Republicans',
 'Republicans have',
 'have a',
 'a new',
 'new fear',
 'fear when',
 'when it',
 'it comes',
 'comes to',
 'to their',
 'their health',
 'health care',
 'care lawsuit',
 'lawsuit against',
 'against the',
 'the Obama',
 'Obama administration',
 'administration :',
 ': They',
 'They might',
 'might win',
 'win .']

In [19]:
libCons.union(conCons)

AttributeError: 'list' object has no attribute 'union'

In [23]:
def extractContent(row):
    sentList = []
    label = bias_dict[row.publication]
    try:
        sentences = sent_tokenize(row.content)
    except:
        sentences = []
    sentences.append(row.title)
    for s in sentences:
        try:
            words = word_tokenize(s)
            words = [w.lower() for w in words]
            bigrams = []
            for i in range(len(words) - 1):
                bigrams.append(' '.join((words[i], words[i+1])))
        except:
            words = []
        if label == 'liberal':
            lexPresent = set(words).intersection(libCons)
            gramsPresent = set(bigrams).intersection(libGrams)
            if lexPresent or gramsPresent:
                sentList.append((label, s))
        elif label == 'conservative':
            lexPresent = set(words).intersection(conCons)
            gramsPresent = set(bigrams).intersection(conGrams)
            if lexPresent or gramsPresent:
                sentList.append((label, s))
        else:
            lexPresent = set(words).intersection(set(libCons).union(conCons))
            gramsPresent = set(bigrams).intersection(set(libGrams).union(conGrams))
            if not lexPresent and not gramsPresent:
                sentList.append((label, s))
    return sentList

In [24]:
sample = df.loc[df.publication == 'Breitbart'].iloc[10]
sample

Unnamed: 0                                                  7813
id                                                         26549
title          New Orleans Removes Famous ‘Lee Circle’ Monume...
publication                                            Breitbart
author                                               John Binder
date                                                  2017-05-20
year                                                        2017
month                                                          5
url                                                          NaN
content        NEW ORLEANS, Louisiana  —   The monument to Ge...
Name: 7813, dtype: object

In [38]:
sentList = []
sample = df.sample().iloc[0]

In [39]:
sentList = extractContent(sample)
len(sentList), len(sample.content)

(6, 3514)

In [40]:
for i,s in enumerate(sentList):
    print(i,s)

0 ('conservative', 'Trailing   in the first set, but leading   on Young’s serve, Medvedev thought he’d hit a winner down the line and walked to his seat believing he had just broken serve, only to realize the chair umpire had called the forehand wide.')
1 ('conservative', 'After a lengthy discussion with Medvedev, Crossland walked over to French in the umpire’s chair and appeared to give his consent for the match to be handed to Young through an unsportsmanlike default against Medvedev.')
2 ('conservative', 'The New York Times compared the incident to Lleyton Hewitt’s infamous comments during a   win against American James Blake at the 2001 US Open  —   which Hewitt went on to win by defeating tennis legend Pete Sampras in the final.')
3 ('conservative', 'Incident echoes Lleyton Hewitt in 2001, who accused a black line judge of favoring James Blake due to ”similarity.”   —   Ben Rothenberg (@BenRothenberg)  Hewitt, then 20, denied any wrongdoing when he was accused of drawing attention

In [136]:
for i,s in enumerate(sent_tokenize(sample.content)):
    print(i,s)

0 NEW ORLEANS, Louisiana  —   The monument to General Robert E. Lee was removed from its pedestal in Lee Circle at the heart of the city of New Orleans.
1 Mayor Mitch Landrieu used the event to claim victory for political correctness.
2 [The famous Civil   monument of Robert E. Lee has been at the center of the city’s most famous traffic circle for more than a century.
3 The statue was put on the chopping block my Landrieu after he began a crusade against four monuments he deemed “symbols of white supremacy.
4 ”  Other monuments removed by the Landrieu Administration include the Battle at Liberty Place, Jefferson Davis, and the P. G. T. Beauregard Monuments, Breitbart Texas reported.
5 All the monuments were removed at some expense to taxpayers, as New Orleans Police Department (NOPD) and New Orleans Fire Department (NOFD) officials were present at each of the removal processes.
6 The money to physically remove the monuments, though, came from a private donor which Landrieu has refused

In [145]:
sents = []
s = extractContent(df.iloc[0])
len(s)

13

In [146]:
print(len(sents))
sents.extend(s)
print(len(sents))

0
13


In [149]:
sents = []
for i in range(20):
    row = df.iloc[i]
    sentList = extractContent(row)
    sents.extend(sentList)
df2 = pd.DataFrame(sents)
df2.columns = ['label', 'sentence']
df2.shape

(44, 2)

In [150]:
df2.head()

,label,sentence
0,liberal,WASHINGTON — Congressional Republicans have...
1,liberal,The incoming Trump administration could choose...
2,liberal,"In another twist, Donald J. Trump’s administra..."
3,liberal,“Given that this pending litigation involves t...
4,liberal,"“Upon taking office, the Trump administration ..."


In [4]:
def extractSentences(df):
    output = []
    for i in range(df.shape[0]):
        row = df.iloc[i]
        sentList = extractContent(row)
        output.extend(sentList)
    df = pd.DataFrame(output)
    df.columns = ['label', 'text']
    return df

In [10]:
start = time.time()
df = extractSentences(df)
print('runtime (min): ', (time.time() - start) / 60)
df.sample(5)

runtime (min):  21.079082457224526


,label,text
276171,neutral,”It’s a gut check about the health of the cons...
101859,conservative,“To me it’s [about] illegal immigration.
193337,neutral,Jay Y. Lee was named as a suspect on Wednesday...
407326,neutral,Its shares rose 2.
381828,neutral,Obama praised Merkel for her ”steady leadershi...


In [11]:
df.shape

(546544, 2)

In [16]:
df.groupby('label').count()

,text
label,
conservative,59668
liberal,181248
neutral,305627


In [12]:
!pwd

/home/alex/Documents/MIDS/w266/final_project/w266_final_project


In [17]:
df.to_csv('filtered_sentences.csv')

In [18]:
import pickle

In [21]:
?open

Signature:
open(
    file,
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
    closefd=True,
    opener=None,
)
Docstring:
Open file and return a stream.  Raise OSError upon failure.

file is either a text or byte string giving the name (and the path
if the file isn't in the current working directory) of the file to
be opened or an integer file descriptor of the file to be
wrapped. (If a file descriptor is given, it is closed when the
returned I/O object is closed, unless closefd is set to False.)

mode is an optional string that specifies the mode in which the file
is opened. It defaults to 'r' which means open for reading in text
mode.  Other common values are 'w' for writing (truncating the file if
it already exists), 'x' for creating and writing to a new file, and
'a' for appending (which on some Unix systems, means that all writes
append to the end of the file regardless of the current seek position).
In text mode, if encoding is not specified

In [26]:
start = time.time()
with open('filtered_sentences.pickle', 'wb') as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)
with open('filtered_sentences.pickle', 'rb') as f:
    df2 = pickle.load(f)
time.time() - start

0.7255961894989014

In [27]:
start = time.time()
df.to_csv('filtered_sentences.csv')
df3 = pd.read_csv('filtered_sentences.csv')
time.time() - start

4.2353599071502686

In [24]:
with open('filtered_sentences.pickle', 'rb') as f:
    df2 = pickle.load(f)

In [25]:
df2.head()

,label,text
0,liberal,WASHINGTON — Congressional Republicans have...
1,liberal,The incoming Trump administration could choose...
2,liberal,"In another twist, Donald J. Trump’s administra..."
3,liberal,“Given that this pending litigation involves t...
4,liberal,"“Upon taking office, the Trump administration ..."
